In [ ]:
import tensorflow as tf 
import numpy as np 
import os 
import time

In [ ]:
with open('/content/beyond_good_and_evil.txt') as f:
  text = f.read()

text = text.lower()

In [ ]:
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

# Take a look at the first 250 characters in text
print(text[:250])

Length of text: 396052 characters



chapter i. prejudices of philosophers


1. the will to truth, which is to tempt us to many a hazardous
enterprise, the famous truthfulness of which all philosophers have
hitherto spoken with respect, what questions has this will to truth not
laid 


In [ ]:
# Find the unique characters in text to build vocab
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

57 unique characters


In [ ]:
# Create a mapping between unique characters and indices. Essentially a 'look up' table to move between characters and indices

char_to_idx = {u:i for i,u in enumerate(vocab)}
idx_to_char = np.array(vocab)

# Represent each character in text with an integer equivalent
text_as_int = np.array([char_to_idx[c] for c in text])

In [ ]:
# Characters with their corresponding index value
print("{")
for char, _ in zip(char_to_idx, range(20)):
  print(" {:4s}: {:3d}, ".format(repr(char), char_to_idx[char]))
print("  ...\n}")

print('{} --- characters mapped to int ---- {}'.format(repr(text[:10]), text_as_int[:10]))

{
 '\n':   0, 
 ' ' :   1, 
 '!' :   2, 
 '"' :   3, 
 '$' :   4, 
 '%' :   5, 
 "'" :   6, 
 '(' :   7, 
 ')' :   8, 
 '*' :   9, 
 ',' :  10, 
 '-' :  11, 
 '.' :  12, 
 '/' :  13, 
 '0' :  14, 
 '1' :  15, 
 '2' :  16, 
 '3' :  17, 
 '4' :  18, 
 '5' :  19, 
  ...
}
'\n\n\nchapter' --- characters mapped to int ---- [ 0  0  0 33 38 31 46 50 35 48]


In [ ]:
# Divde text in example sequence, each input will contain seq_length characters 
# For each input seq, the corresponding target contains same length, but shifted one char to right
# The maximum length sentence you want for a single input in characters
seq_length = 100 
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets 
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(20):
  print(idx_to_char[i.numpy()])







c
h
a
p
t
e
r
 
i
.
 
p
r
e
j
u
d


In [ ]:
# Batch method lets us convert these induvidual characters to sequences of desired size

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx_to_char[item.numpy()])))

'\n\n\nchapter i. prejudices of philosophers\n\n\n1. the will to truth, which is to tempt us to many a hazar'
'dous\nenterprise, the famous truthfulness of which all philosophers have\nhitherto spoken with respect,'
' what questions has this will to truth not\nlaid before us! what strange, perplexing, questionable que'
'stions! it is\nalready a long story; yet it seems as if it were hardly commenced. is\nit any wonder if '
'we at last grow distrustful, lose patience, and turn\nimpatiently away? that this sphinx teaches us at'


In [ ]:
# For each sequence, duplicate and shift it to form the input and target text by using the map method

def split_input_target (chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

# print first example of input and target values

for input_example, target_example in dataset.take(1):
  print('Input data:  ', repr(''.join(idx_to_char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx_to_char[target_example.numpy()])))

Input data:   '\n\n\nchapter i. prejudices of philosophers\n\n\n1. the will to truth, which is to tempt us to many a haza'
Target data:  '\n\nchapter i. prejudices of philosophers\n\n\n1. the will to truth, which is to tempt us to many a hazar'


In [ ]:
# Create training batches by splitting data in managable sequences

batch_size = 64
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder = True)

dataset



<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Build Model 

# Length of vocab in characters
vocab_size = len(vocab)

# the embedding dimensions
embedding_dim = 256

#number of RNN units
rnn_units = 1024

def build_model (vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                                batch_input_shape = [batch_size, None]),
      tf.keras.layers.LSTM(rnn_units, 
                          return_sequences = True, 
                          stateful = True, 
                          recurrent_initializer = 'glorot_uniform'),
      tf.keras.layers.Dropout(0.2), 
      tf.keras.layers.LSTM(rnn_units, 
                          return_sequences = True, 
                          stateful = True, 
                          recurrent_initializer = 'glorot_uniform'), 
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(vocab_size)                        
  ])
  return model 

model = build_model(
    vocab_size = len(vocab), 
    embedding_dim = embedding_dim, 
    rnn_units = rnn_units, 
    batch_size = batch_size)

In [ ]:
# Test to see if model will run as expected 

for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, '# (batch_size, sequence_length, vocab_size)')

(64, 100, 57) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           14592     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dropout (Dropout)            (64, None, 1024)          0         
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dropout_1 (Dropout)          (64, None, 1024)          0         
_________________________________________________________________
dense (Dense)                (64, None, 57)            58425     
Total params: 13,712,697
Trainable params: 13,712,697
Non-trainable params: 0
____________________________________________

In [ ]:
# Train model using adam optimizer and sparse_catergorical_crossentropy

# Because our model returns logits we need to set the from_logits flat

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

model.compile(optimizer = 'adam', loss = loss)

Prediction shape:  (64, 100, 57)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.0428247


In [ ]:
# Configure checkpoints to ensure checkpoints are saved during training 

# Directory where checkpoints will be saved 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpont_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix, 
    save_weights_only = True)

In [ ]:
# Execute training 

epochs = 20

history = model.fit(dataset, epochs=epochs, callbacks=[checkpont_callback])

Epoch 1/20
61/61 [==============================] - 1096s 18s/step - loss: 3.0835
Epoch 2/20
61/61 [==============================] - 1108s 18s/step - loss: 2.5913
Epoch 3/20
61/61 [==============================] - 1088s 18s/step - loss: 2.2143
Epoch 4/20
61/61 [==============================] - 1096s 18s/step - loss: 1.9439
Epoch 5/20
61/61 [==============================] - 1095s 18s/step - loss: 1.7389
Epoch 6/20
61/61 [==============================] - 1085s 18s/step - loss: 1.5978
Epoch 7/20
61/61 [==============================] - 1083s 18s/step - loss: 1.4980
Epoch 8/20
61/61 [==============================] - 1079s 18s/step - loss: 1.4228
Epoch 9/20
61/61 [==============================] - 1065s 17s/step - loss: 1.3645
Epoch 10/20
61/61 [==============================] - 1059s 17s/step - loss: 1.3098
Epoch 11/20
61/61 [==============================] - 1084s 18s/step - loss: 1.2597
Epoch 12/20
61/61 [==============================] - 1076s 18s/step - loss: 1.2121
Epoch 13/20
6

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_20'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 100

    # Converting our start string to numbers (vectorizing)
    input_eval = [char_to_idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx_to_char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"the"))

ther
wich genaine and neighbour inclinations; but there is a common
ourselves!
this isuro impeas my str


In [ ]:
while True: pass

In [ ]:
def save_model(model, suffix=None):
  """
  Saves a given model in a models directory and appends a suffix (str)
  for clarity and reuse.
  """
  # Create model directory with current time
  modeldir = os.path.join("/content/drive/My Drive/Colab Notebooks/Existential_RNN/")
  model_path = modeldir + "-" + suffix + ".h5" # save format of model
  print(f"Saving model to: {model_path}...")
  model.save(model_path)
  return model_path

In [ ]:
save_model(model, suffix="Double_LTSM_20_Epochs")

Saving model to: /content/drive/My Drive/Colab Notebooks/Existential_RNN/-Double_LTSM_20_Epochs.h5...


'/content/drive/My Drive/Colab Notebooks/Existential_RNN/-Double_LTSM_20_Epochs.h5'